**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [ ]:
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest

**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

**Text Preprocessing**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

In [ ]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [ ]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [ ]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def nGram(train_text, test_text):
  #unique word and word count
  vectorizer = TfidfVectorizer(ngram_range=(1,7))
  
  X1 = vectorizer.fit_transform(train_text)
  word_list = vectorizer.get_feature_names()
  word_matrix1 = X1.toarray()
  count_list1 = word_matrix1.sum(axis=0)
  A = pd.DataFrame(word_matrix1, columns = word_list)


  X2 = vectorizer.transform(test_text)
  word_list = vectorizer.get_feature_names()
  word_matrix2 = X2.toarray()
  count_list2 = word_matrix2.sum(axis=0)
  B = pd.DataFrame(word_matrix2, columns = word_list)
  return A, B
  

In [ ]:
X_train, X_test = nGram(Train_data['cleanText'][0:1000], Val_data['cleanText'][0:1000])

#y_train = Train_data['label'][0:1000]
#y_test = Val_data['label'][0:1000]

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(Train_data['label'][0:1000])
y_test = label_encoder.transform(Val_data['label'][0:1000])
label_encoder.classes_

array(['fake', 'real'], dtype=object)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1000, 72197), (1000, 72197), (1000,), (1000,))

In [ ]:
X_train

,_allowances,_allowances members,_allowances members parliament,_allowances members parliament india,_allowances members parliament india increased,_allowances members parliament india increased thousand,_allowances members parliament india increased thousand rupees,_an,_an image,_an image shared,_an image shared hundreds,_an image shared hundreds times,_an image shared hundreds times multiple,_an image shared hundreds times multiple facebook,_bill,_bill gates,_bill gates coronavirus,_bill gates coronavirus vaccine,_bill gates coronavirus vaccine developed,_bill gates coronavirus vaccine developed oxford,_bill gates coronavirus vaccine developed oxford researchers,_israel,_israel found,_israel found cure,_israel found cure vaccine,_israel found cure vaccine coronavirus,_photo,_photo vaccine,_photo vaccine developed,_photo vaccine developed scientists,_photo vaccine developed scientists covid,_while,_while western,_while western countries,_while western countries busy,_while western countries busy fighting,_while western countries busy fighting new,_while western countries busy fighting new coronavirus,aaj,aaj tak,...,zithromax know people,zithromax know people want,zithromax know people want talk,zithromax know people want talk mask,zithromax know people want talk mask hello,zomato,zomato deepinder,zomato deepinder made,zomato deepinder made big,zomato deepinder made big deal,zomato deepinder made big deal someone,zomato deepinder made big deal someone refusing,zones,zones july,zones july covid,zones july covid coronavirusfacts,zones strict,zones strict enforcement,zones strict enforcement lockdown,zones strict enforcement lockdown containment,zones strict enforcement lockdown containment zones,zones strict enforcement lockdown containment zones till,zones till,zones till september,zones till september details,zones till september details staysafe,zones till september details staysafe indiawillwin,zoo,zoo pandas,zookeepers,zookeepers novel,zookeepers novel coronavirus,zookeepers novel coronavirus pandemic,zoom,zoom topic,zoom topic covid,zoom topic covid amp,zoom topic covid amp telehealth,zoom topic covid amp telehealth implementation,zoom topic covid amp telehealth implementation stories
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
X_test

,_allowances,_allowances members,_allowances members parliament,_allowances members parliament india,_allowances members parliament india increased,_allowances members parliament india increased thousand,_allowances members parliament india increased thousand rupees,_an,_an image,_an image shared,_an image shared hundreds,_an image shared hundreds times,_an image shared hundreds times multiple,_an image shared hundreds times multiple facebook,_bill,_bill gates,_bill gates coronavirus,_bill gates coronavirus vaccine,_bill gates coronavirus vaccine developed,_bill gates coronavirus vaccine developed oxford,_bill gates coronavirus vaccine developed oxford researchers,_israel,_israel found,_israel found cure,_israel found cure vaccine,_israel found cure vaccine coronavirus,_photo,_photo vaccine,_photo vaccine developed,_photo vaccine developed scientists,_photo vaccine developed scientists covid,_while,_while western,_while western countries,_while western countries busy,_while western countries busy fighting,_while western countries busy fighting new,_while western countries busy fighting new coronavirus,aaj,aaj tak,...,zithromax know people,zithromax know people want,zithromax know people want talk,zithromax know people want talk mask,zithromax know people want talk mask hello,zomato,zomato deepinder,zomato deepinder made,zomato deepinder made big,zomato deepinder made big deal,zomato deepinder made big deal someone,zomato deepinder made big deal someone refusing,zones,zones july,zones july covid,zones july covid coronavirusfacts,zones strict,zones strict enforcement,zones strict enforcement lockdown,zones strict enforcement lockdown containment,zones strict enforcement lockdown containment zones,zones strict enforcement lockdown containment zones till,zones till,zones till september,zones till september details,zones till september details staysafe,zones till september details staysafe indiawillwin,zoo,zoo pandas,zookeepers,zookeepers novel,zookeepers novel coronavirus,zookeepers novel coronavirus pandemic,zoom,zoom topic,zoom topic covid,zoom topic covid amp,zoom topic covid amp telehealth,zoom topic covid amp telehealth implementation,zoom topic covid amp telehealth implementation stories
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
y_train

array([1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,

In [ ]:
y_test

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1000, 72197), (1000, 72197), (1000,), (1000,))

**Model Evaluation**

In [ ]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    "MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gaussian Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [ ]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [ ]:
def batch_classify(dict_classifiers, X_train,X_test,y_train,y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        y_pred = classifier.predict(X_test)
        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [ ]:
dict_super_models = batch_classify(dict_classifiers, X_train,X_test,y_train,y_test)

Nearest Neighbors
Accuracy: 0.813000
Precision: 0.816250
Recall: 0.813000
F1 score: 0.813121
Cohens kappa: 0.626606
[[399  72]
 [115 414]]
train score:{0.90} test_score:{0.81}
LogisticRegression
Accuracy: 0.838000
Precision: 0.843168
Recall: 0.838000
F1 score: 0.836590
Cohens kappa: 0.672313
[[356 115]
 [ 47 482]]
train score:{1.00} test_score:{0.84}
Decision Tree
Accuracy: 0.674000
Precision: 0.753394
Recall: 0.674000
F1 score: 0.654462
Cohens kappa: 0.365732
[[443  28]
 [298 231]]
train score:{0.72} test_score:{0.67}
Random Forest
Accuracy: 0.526000
Precision: 0.413721
Recall: 0.526000
F1 score: 0.368194
Cohens kappa: -0.005503
[[  2 469]
 [  5 524]]
train score:{0.57} test_score:{0.53}
MLP
Accuracy: 0.867000
Precision: 0.867946
Recall: 0.867000
F1 score: 0.866630
Cohens kappa: 0.732136
[[389  82]
 [ 51 478]]
train score:{1.00} test_score:{0.87}
AdaBoost
Accuracy: 0.806000
Precision: 0.813234
Recall: 0.806000
F1 score: 0.805868
Cohens kappa: 0.613747
[[408  63]
 [131 398]]
train scor

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Accuracy: 0.492000
Precision: 0.755600
Recall: 0.492000
F1 score: 0.346384
Cohens kappa: 0.037481
[[471   0]
 [508  21]]
train score:{1.00} test_score:{0.49}
GradientBoost
Accuracy: 0.823000
Precision: 0.830217
Recall: 0.823000
F1 score: 0.822893
Cohens kappa: 0.647554
[[416  55]
 [122 407]]
train score:{0.94} test_score:{0.82}
MultinomialNB
Accuracy: 0.836000
Precision: 0.861854
Recall: 0.836000
F1 score: 0.831387
Cohens kappa: 0.665520
[[320 151]
 [ 13 516]]
train score:{1.00} test_score:{0.84}
SGD
Accuracy: 0.869000
Precision: 0.869674
Recall: 0.869000
F1 score: 0.868697
Cohens kappa: 0.736287
[[392  79]
 [ 52 477]]
train score:{1.00} test_score:{0.87}
LGBM
Accuracy: 0.795000
Precision: 0.798729
Recall: 0.795000
F1 score: 0.795106
Cohens kappa: 0.590854
[[392  79]
 [126 403]]
train score:{0.95} test_score:{0.80}
Gaussian Process Classifier
Accuracy: 0.866000
Precision: 0.867017
Recall: 0.866000
F1 score: 0.865611
Cohens kappa: 0.730090
[[388  83]
 [ 51 478]]
train score:{1.00} test_

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train score:{0.54} test_score:{0.53}
Support Vector Machine 2
Accuracy: 0.725000
Precision: 0.801166
Recall: 0.725000
F1 score: 0.700285
Cohens kappa: 0.431127
[[205 266]
 [  9 520]]
train score:{1.00} test_score:{0.72}


In [ ]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [ ]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
11,SGD,0.869,0.869674,0.869,0.868697,0.736287
4,MLP,0.867,0.867946,0.867,0.866630,0.732136
13,Gaussian Process Classifier,0.866,0.867017,0.866,0.865611,0.730090
6,GaussianNB,0.862,0.863148,0.862,0.861565,0.721968
1,LogisticRegression,0.838,0.843168,0.838,0.836590,0.672313
10,MultinomialNB,0.836,0.861854,0.836,0.831387,0.665520
9,GradientBoost,0.823,0.830217,0.823,0.822893,0.647554
0,Nearest Neighbors,0.813,0.816250,0.813,0.813121,0.626606
5,AdaBoost,0.806,0.813234,0.806,0.805868,0.613747
12,LGBM,0.795,0.798729,0.795,0.795106,0.590854


In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &  Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &     0.813 &   0.816250 &   0.813 &  0.813121 &  0.626606 \\
          LogisticRegression &     0.838 &   0.843168 &   0.838 &  0.836590 &  0.672313 \\
               Decision Tree &     0.674 &   0.753394 &   0.674 &  0.654462 &  0.365732 \\
               Random Forest &     0.526 &   0.413721 &   0.526 &  0.368194 & -0.005503 \\
                         MLP &     0.867 &   0.867946 &   0.867 &  0.866630 &  0.732136 \\
                    AdaBoost &     0.806 &   0.813234 &   0.806 &  0.805868 &  0.613747 \\
                  GaussianNB &     0.862 &   0.863148 &   0.862 &  0.861565 &  0.721968 \\
          LinearDiscriminant &     0.697 &   0.782030 &   0.697 &  0.664389 &  0.371395 \\
       QuadraticDiscriminant &     0.492 &   0.755600 &   0.492 &  0.346384 &  0.037481 \\
               GradientBoost &     0.823 &   0.8